# Filtering log data with RePlay

This notebook presents the RePlay functionality for log data filtering by interaction and time.<br>
You can conveniently filter your datasets, for example, leave only positive reviews, select the desired period of time, leave only active users
<br>RePlay offers filtering functions:

* filter_by_min_count - filtering by the number of interactions
* filter_out_low_ratings - filtering by the rating
* take_num_user_interactions - filtering by the number of interactions for each user's log
* take_num_days_of_global_hist - filtering by the time period
* take_num_days_of_user_hist - filtering by the time period for each user's log
* take_time_period - filtering by the selected date


In [ ]:
! pip install rs-datasets -q

In [1]:
import pandas as pd
from rs_datasets import MovieLens

from replay.data_preparator import DataPreparator, Indexer
from replay.session_handler import State
from rs_datasets import MovieLens
from replay.utils import get_log_info, convert2spark
from pyspark.sql import functions as sf
from pyspark.sql.types import TimestampType

from replay.filters import (
    filter_by_min_count,
    filter_out_low_ratings,
    take_num_user_interactions,
    take_num_days_of_user_hist,
    take_time_period,
    take_num_days_of_global_hist
)

In [3]:
spark = State().session
spark.sparkContext.setLogLevel('ERROR')
spark

## Get started

Download the dataset **MovieLens**

In [4]:
data = MovieLens("1m")
log = data.ratings

Use the `convert2spark` function to convert pandas.DataFrame to spark.DataFrame

In [5]:
log_spark = convert2spark(log)

In [7]:
log_spark.show(4)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|   1193|     5|978300760|
|      1|    661|     3|978302109|
|      1|    914|     3|978301968|
|      1|   3408|     4|978300275|
+-------+-------+------+---------+
only showing top 4 rows



## function filter_by_min_count()

Filtering by the minimum number of interactions.

Parameters:

* `data_frame` - DataFrame of interaction between users and elements

* `num_entries` - threshold value for the number of interactions to filter, includes a specified threshold
    
* `group_by` - column relative to which filtering occurs
    

The picture shows how filtering works using the example of a simple interaction log.<br>Colored cells are user relevance for the item. The color depends on the relevance set

![title](img/filter_by_min_count.jpg)

Let's leave users who have `>= 50` interactions with items

In [8]:
log_filter = filter_by_min_count(log_spark, 
                                 num_entries=50,
                                 group_by="user_id")

28-Dec-22 18:40:21, replay, INFO: current threshold removes 0.0567261442358547% of data


In [9]:
log_filter.groupBy("user_id").count().select(sf.min("count").alias("Minimum number of interactions per user's")).show()

+-----------------------------------------+
|Minimum number of interactions per user's|
+-----------------------------------------+
|                                       50|
+-----------------------------------------+



Number of users has decreased

In [10]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 943471, total users: 4297, total items: 3689


Let's leave the items that `>= 100` users interacted with

In [11]:
log_filter = filter_by_min_count(log_spark, 
                                 num_entries=100,
                                 group_by="item_id")

28-Dec-22 18:40:32, replay, INFO: current threshold removes 0.05797188387626986% of data


In [12]:
log_filter.groupBy("item_id").count().select(sf.min("count").alias("Minimum number of interactions per item's")).show()

+-----------------------------------------+
|Minimum number of interactions per item's|
+-----------------------------------------+
|                                      100|
+-----------------------------------------+



Number of items has decreased

In [13]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 942225, total users: 6040, total items: 2019


## function filter_out_low_ratings()

Filtering relevance with a minimum threshold.

Parameters:

* `data_frame` - DataFrame of interaction between users and elements
    
* `value` - threshold value of the relevance for filtering, includes a specified threshold

* `rating_column` - relevance column
    

![title](img/filter_by_min_count.jpg)

Let's leave interactions with relevance `5`

In [14]:
log_filter = filter_out_low_ratings(log_spark,
                                    value=5,
                                    rating_column="rating")

In [15]:
log_filter.select(sf.min("rating").alias("Minimum rating")).show()

+--------------+
|Minimum rating|
+--------------+
|             5|
+--------------+



The number of users and interactions has decreased

In [16]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 226310, total users: 6014, total items: 3232


## function take_num_user_interactions()

Filtering the number of interactions for each user. The order of interaction is determined by the date.

Parameters:


* `log` - DataFrame of interaction between users and elements
    
* `num_interactions` - threshold value for the number of interactions for each user
    
* `first` - indicator, if the value is `true`,  counting of user's interactions start from the beginning, if the value is `false`, the counting of interactions start from the end
    
* `date_col` - date column name
    
* `user_col` - user id column name
    
* `item_col` - item id column name


![title](img/take_num_user_interactions.jpg)

Let's leave the last `2` interactions for each user

In [17]:
log_filter = take_num_user_interactions(log_spark, 
                                        num_interactions=2,
                                        first=False,
                                        user_col="user_id",
                                        item_col="item_id")

In [20]:
log_filter.groupBy("user_id").count().select(sf.max("count").alias("Maximun number of interactions per user's")).show()

+-----------------------------------------+
|Maximun number of interactions per user's|
+-----------------------------------------+
|                                        2|
+-----------------------------------------+



Note that the number of users has not changed

In [21]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 12080, total users: 6040, total items: 2430


## function take_num_days_of_user_hist()

Filtering by the time period of interactions for each user.

Parameters:

* `log` - DataFrame of interaction between users and elements
    
* `days` - time interval in days
    
* `first` - indicator, if the value is `true`, the interval starts counting from the first user's interaction, if the value is `false`, the interval starts counting from the last interaction
    
* `date_col` - item id column name
    
* `user_col` - user id column name

![title](img/take_num_days_of_user_hist.jpg)

Let's leave the last `week` of each user's interactions

In [22]:
log_filter = take_num_days_of_user_hist(log_spark, 
                                        days=7,
                                        first=True,
                                        date_col="timestamp",
                                        user_col="user_id")

In [24]:
(
    log_filter
    .groupBy("user_id")
    .agg((sf.max("timestamp") - sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("max_period"))
    .withColumn("Maximum difference in days per user's",sf.round(sf.col("max_period")/(24*3600)))
    .select("Maximum difference in days per user's")
).show()

+-------------------------------------+
|Maximum difference in days per user's|
+-------------------------------------+
|                                  7.0|
+-------------------------------------+



The number of users has not changed, but the number of interactions has decreased

In [25]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 818263, total users: 6040, total items: 3660


## function take_num_days_of_global_hist()

Filtering by the time period of interactions of all users.

Parameters:

* `log` - DataFrame of interaction between users and elements
    
* `duration_days` - time interval in days
    
* `first` - indicator, if the value is `true`, the interval starts counting from the first interaction among all users, if the value is `false`, the interval starts counting from the last interaction
    
* `date_column` - date column name

![title](img/take_num_days_of_global_hist.jpg)

Let's leave the last `2 weeks`of interactions

In [26]:
log_filter = take_num_days_of_global_hist(log_spark, 
                                          duration_days=14,
                                          first=False)

In [27]:
(
    log_filter
    .agg((sf.max("timestamp") - sf.min("timestamp")).alias("diff"))
    .select(sf.max("diff").alias("max_period"))
    .withColumn("Maximum difference in days among all interactions",sf.round(sf.col("max_period")/(24*3600)))
    .select("Maximum difference in days among all interactions")
).show()

+-------------------------------------------------+
|Maximum difference in days among all interactions|
+-------------------------------------------------+
|                                             14.0|
+-------------------------------------------------+



After such filtering, the number of interactions and users significantly decreased

In [28]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 597, total users: 66, total items: 501


## function take_time_period()

Filtering by a given time period.

Parameters:
    
* `log` - DataFrame of interaction between users and elements
    
* `start_date` - beginning of the time period
    
* `end_date` - end of time period
    
* `date_column` - date column name

![title](img/take_time_period.jpg)

Let's leave the data for `March` only

In [29]:
log_filter = take_time_period(log_spark, 
                              start_date="2001-03-01",
                              end_date="2001-04-01")

In [30]:
(
    log_filter
    .select(
        sf.min("timestamp").cast(TimestampType()).alias("Minimum date"),
        sf.max("timestamp").cast(TimestampType()).alias("Maximum date")
    )                            
).show()

+-------------------+-------------------+
|       Minimum date|       Maximum date|
+-------------------+-------------------+
|2001-03-01 03:08:52|2001-03-31 23:12:22|
+-------------------+-------------------+



The number of records has noticeably decreased

In [31]:
print("Before filtering\n", get_log_info(log_spark, user_col="user_id", item_col="item_id"))
print("After filtering\n", get_log_info(log_filter, user_col="user_id", item_col="item_id"))

Before filtering
 total lines: 1000209, total users: 6040, total items: 3706
After filtering
 total lines: 5986, total users: 321, total items: 2010
